In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.basics import *
from fastcore.foundation import *
from fastcore.test import *
from fastcore.script import call_parse, Param

from pathlib import Path
import re, time
import threading

from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler


class MarkdownHandler(FileSystemEventHandler):
    """Handles markdown file events and extracts code blocks"""
    def __init__(self, output_dir='py'):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
    
    def should_process(self, path):
        "Check if the markdown file should be processed"
        return path.suffix == '.md' and path.name.lower() != 'readme.md'
    
    def on_modified(self, event):
        path = Path(event.src_path)
        if not self.should_process(path): return
        self.process_markdown(path)
        
    def on_created(self, event):
        path = Path(event.src_path)
        if not self.should_process(path): return
        self.process_markdown(path)
        
    def process_markdown(self, md_path):
        "Process a markdown file and save extracted code"
        content = md_path.read_text()
        code = self.extract_code(content)
        if code:
            py_path = self.output_dir/f"{md_path.stem}.py"
            py_path.write_text(code)

    def extract_code(self, md_content):
        "Extract code blocks from markdown content"
        # Match both single and triple backtick code blocks
        pattern = r'`{3}[\w\s]*\n(.*?)`{3}|`([^`]+)`'
        code_blocks = []
        for match in re.finditer(pattern, md_content, re.DOTALL):
            # Get whichever group matched (triple or single backticks)
            code = match.group(1) or match.group(2)
            if code: code_blocks.append(code)
        return '\n'.join(code_blocks)

In [ ]:
md = MarkdownHandler()

# Test single backtick
test_str = "Some text `print('hello')`"
test_eq(md.extract_code(test_str), "print('hello')")

In [ ]:
# Test triple backtick
test_str = """Some text
```
def hello():
    print('world')
```
more text"""
print(test_str)

Some text
```
def hello():
    print('world')
```
more text


In [ ]:
expected = "def hello():\n    print('world')\n"
print(expected)

def hello():
    print('world')



In [ ]:
test_eq(md.extract_code(test_str), expected)

In [ ]:
# Test multiple code blocks
test_str = """```python
def hello():
    print('world')
```
Some text
```
def goodbye():
    print('bye')
```"""
print(test_str)

```python
def hello():
    print('world')
```
Some text
```
def goodbye():
    print('bye')
```


In [ ]:
expected = "def hello():\n    print('world')\n\ndef goodbye():\n    print('bye')\n"
print(expected)

def hello():
    print('world')

def goodbye():
    print('bye')



In [ ]:
test_eq(md.extract_code(test_str), expected)

test_eq(md.should_process(Path('test.md')), True)
test_eq(md.should_process(Path('test.py')), False)
test_eq(md.should_process(Path('test/test.md')), True)

test_eq(md.should_process(Path('README.md')), False)
test_eq(md.should_process(Path('readme.md')), False)
test_eq(md.should_process(Path('ReadMe.md')), False)

In [ ]:
from tempfile import NamedTemporaryFile

def test_process_markdown(): # Create a temporary markdown file
    with NamedTemporaryFile(suffix='.md', mode='w', delete=False) as f:
        f.write("""# Test file Some markdown text
```
def test_func():
    return 42
```
More text
```
print('hello')```""")
    md_path = Path(f.name)

    # Create handler with temporary output dir
    temp_out = Path('temp_test_output')
    temp_out.mkdir(exist_ok=True)
    md = MarkdownHandler(output_dir=temp_out)

    md.process_markdown(md_path)

    out_file = temp_out/f"{md_path.stem}.py"
    test_eq(out_file.exists(), True)
    expected = "def test_func():\n    return 42\n\nprint('hello')"
    test_eq(out_file.read_text().strip(), expected)
    
    md_path.unlink()
    out_file.unlink()
    temp_out.rmdir()

test_process_markdown()

In [ ]:
#| export

@call_parse
def literati(path:Param("Directory to monitor", str)='.', 
             output_dir:Param("Output directory for Python files", str)='py'):
    "Monitor markdown files and extract code blocks to Python files"
    observer = Observer()
    handler = MarkdownHandler(output_dir=output_dir)
    observer.schedule(handler, path, recursive=False)
    observer.start()
    
    print(f"G'day! Monitoring {path} for markdown files...")
    print(f"Python files will be saved to {output_dir}/")
    print("Optimizing the servo run... (Press Ctrl+C to stop)")
    
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

In [ ]:
def test_literati():
    # Create a simple way to access the observer
    observer_container = []
    
    def run_literati():
        obs = literati(path='.', output_dir='test_py')
        observer_container.append(obs)
    
    # Start literati in a separate thread
    thread = threading.Thread(target=run_literati)
    thread.daemon = True
    thread.start()
    
    # Give it a moment to start
    time.sleep(0.5)
    
    # Verify the output directory was created
    test_eq(Path('test_py').exists(), True)
    
    # Stop the observer properly
    if observer_container:
        observer_container[0].stop()
        observer_container[0].join()
    
    # Cleanup
    Path('test_py').rmdir()

test_literati()

G'day! Monitoring . for markdown files...
Python files will be saved to test_py/
Optimizing the servo run... (Press Ctrl+C to stop)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()